# part one
```
--- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

Your puzzle answer was 521601.
```

In [1]:
with open("input.txt", "r") as f:
    lines = f.readlines()
lines[:10]

['48.................501....33.....622..............763.........331.................161.683......................................980..........\n',
 '...491.842.....948*..................338.....*......=...........-...309.......633*....*....................*990...706...452......*..+.......\n',
 '...*...*....................426........*..408.........................*............659...............250.........&.......*.....767...403....\n',
 '363.....961...959#.508*223......843.406..........690................%..479.....................398.../....*............458..................\n',
 '......=.........................................*....@946........767.........907=.................@....158.850..+670..............10*790....\n',
 '.......39.306...679.%113...............665....874....................597...................861.........................840...855............\n',
 '.................*.........864.....154*....#..............1*......4..........341..................731*..........783.

In [2]:
def is_symbol(char: str):
    return not (char.isdigit() or char in ".\n")

def is_digit(char:str):
    return char.isdigit()


In [3]:
# find position of numbers and symbols (O(nlines*lineslength))
numbers = []
symbols = []
possible_gears = []
for i, line in enumerate(lines):
    char_num = 0
    while char_num < len(line):
        number, start = "", None
        while is_digit(line[char_num]):
            if not number:
                start = char_num
            number += line[char_num]
            char_num += 1

        if number:
            numbers.append((number, i, start))
        
        if is_symbol(line[char_num]):
            symbols.append((i, char_num))
            if line[char_num] == "*":
                possible_gears.append((i, char_num))
        
        char_num += 1


In [4]:
print(f"{numbers[:3]=}, {symbols[:3]=}")
print(f"{len(numbers)=}, {len(symbols)=}")

numbers[:3]=[('48', 0, 0), ('501', 0, 19), ('33', 0, 26)], symbols[:3]=[(1, 18), (1, 45), (1, 52)]
len(numbers)=1207, len(symbols)=740


In [5]:
from typing import Any
import bisect

class SparseMatrix:
    def __init__(self, symbols) -> None:
        self.rows = {}
        for x,y in symbols:
            if self.rows.get(x) is None:
                self.rows[x] = []
            bisect.insort(self.rows[x], y)

    def exists(self, x, y):
        row = self.rows.get(x)
        if row is None:
            return False
        index = bisect.bisect_left(row, y)
        return index < len(row) and row[index] == y

sparse_symbols_matrix = SparseMatrix(symbols)

In [6]:
def get_adjacent_indices(x, y, matrix=lines):
    rows, cols = len(matrix), len(matrix[0])
    adjacent_indices = []

    # Define possible offsets for neighboring cells
    offsets = [(-1, -1), (-1, 0), (-1, 1),
               (0, -1),           (0, 1),
               (1, -1), (1, 0), (1, 1)]

    for dx, dy in offsets:
        new_x, new_y = x + dx, y + dy

        # Check if the new indices are within bounds
        if 0 <= new_x < rows and 0 <= new_y < cols:
            adjacent_indices.append((new_x, new_y))

    return adjacent_indices

adjacent_numbers_sum = 0
for number, row, start_col in numbers:

    appended = False
    for i in range(start_col, start_col+len(number)):
        
        # find adjacent symbols in O(1)
        for X, Y in get_adjacent_indices(row, i):
            if sparse_symbols_matrix.exists(X, Y):
                adjacent_numbers_sum += int(number)
                appended = True
                break
        if appended:
            break
        
print(adjacent_numbers_sum)

521601


# Part two

```
--- Part Two ---
The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

Your puzzle answer was 80694070.


```

In [7]:
class SparseNumberMatrix:
    def __init__(self, numbers) -> None:
        self.numbers = {}
        for val, row, start_col in numbers:
            if self.numbers.get(row) is None:
                self.numbers[row] = []
            self.numbers[row].append((val, start_col))
    
    def get_number_covering_cell(self, x,y):
        row = self.numbers.get(x)
        if row is None:
            return None
        for number_tuple in row:
            number, start_pos = number_tuple
            if start_pos <= y < start_pos + len(number):
                return number_tuple

sparse_number_matrix = SparseNumberMatrix(numbers)


In [8]:
from functools import reduce



result = 0

for gear in possible_gears:
    adjacent_indices = get_adjacent_indices(*gear)

    adjacent_numbers = [sparse_number_matrix.get_number_covering_cell(*pos) for pos in adjacent_indices]
    adjacent_numbers = set([int(x[0]) for x in adjacent_numbers if x])
    
    
    if len(adjacent_numbers) > 1:
        result += reduce(lambda x,y: x*y, adjacent_numbers, 1)
    

print(result)


80694070
